In [11]:

%pip install requests beautifulsoup4 pillow gradio


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: C:\Users\91936\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [3]:
import gradio as gr
import requests
from bs4 import BeautifulSoup
import io
import base64
from PIL import Image
import re


In [32]:
API_URL = "http://localhost:1234/v1/chat/completions"

# Extract images from a webpage
def extract_images_from_url(url):
    try:
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        images = [img['src'] for img in soup.find_all('img') if 'src' in img.attrs]
        return images[:3]  # Limit to 3 images
    except Exception as e:
        return []

# Extract text from a webpage
def extract_text_from_url(url):
    try:
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        elements = soup.find_all(['p', 'li', 'div', 'span'])
        text = " ".join([el.get_text() for el in elements if el.get_text(strip=True)])
        return text if text else "No text found."
    except Exception as e:
        return "Failed to extract text."

# Encode an image
def encode_image(image_url):
    try:
        response = requests.get(image_url, stream=True)
        return response.content if response.status_code == 200 else None
    except Exception as e:
        return None

# Generate image captions with user prompt
def generate_caption(image_data, user_prompt="Describe this image with key points"):
    if not image_data:
        return "Failed to process image."
    
    try:
        image = Image.open(io.BytesIO(image_data))
        buffered = io.BytesIO()
        image.save(buffered, format="JPEG", quality=85)
        base64_image = base64.b64encode(buffered.getvalue()).decode("utf-8")

        payload = {
            "model": "llava",
            "messages": [{"role": "user", "content": [
                {"type": "text", "text": user_prompt},
                {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}}
            ]}],
            "temperature": 0.7
        }

        response = requests.post(API_URL, json=payload)
        return response.json()["choices"][0]["message"]["content"]
    except Exception as e:
        return "Error generating caption."

# Summarize extracted text
def summarize_text(text):
    if not text:
        return "No text found to summarize."
    
    text = text[:2000] if len(text) > 2000 else text
    
    payload = {
        "model": "slim-summary-phi-3",
        "messages": [
            {"role": "system", "content": "You are an expert summarizer. Create concise, accurate summaries."},
            {"role": "user", "content": f"Summarize the following text:\n\n{text}"}
        ],
        "temperature": 0.3,
        "max_tokens": 500
    }
    
    try:
        response = requests.post(API_URL, json=payload)
        response.raise_for_status()
        return response.json()["choices"][0]["message"]["content"]
    except Exception as e:
        return f"Summarization failed: {str(e)}"

# Process a URL
def process_url(url, user_prompt="Describe this image with key points"):
    images = extract_images_from_url(url)
    text = extract_text_from_url(url)
    image_data = [encode_image(img) for img in images if encode_image(img)]
    captions = [generate_caption(img_data, user_prompt) for img_data in image_data]
    summary = summarize_text(text)
    return images, captions, summary

# Extract URL and optional caption prompt
def extract_url_and_prompt(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+|\S+\.(com|org|edu|gov|io)\S*')
    match = url_pattern.search(text)
    
    url = match.group(0) if match else None
    prompt_match = re.search(r'caption:\s*(.*)', text)
    caption_prompt = prompt_match.group(1).strip() if prompt_match else "Describe this image with key points"
    
    return url, caption_prompt

# Chatbot logic
def chatbot(message, history):
    url, user_prompt = extract_url_and_prompt(message)
    response = ""

    if url:
        images, captions, summary = process_url(url, user_prompt)
        response = f"**Summary of {url}:**\n\n{summary}\n\n"

        if images:
            response += "**Image Captions:**\n\n"
            for i, caption in enumerate(captions):
                response += f"**Image {i+1}:** {caption}\n\n"

    elif "help" in message.lower():
        response = "**How to use:**\n1. Send a webpage URL\n2. Add 'caption: [your prompt]' for custom captions\n3. I extract text/images, summarize & caption images\n\nExample: 'Summarize https://example.com caption: Explain the diagram in detail'"
    else:
        response = "I'm here to summarize web content. Please provide a URL."
    
    history.append((message, response))
    return history, ""

# Gradio UI
with gr.Blocks(css="footer {visibility: hidden}") as iface:
    gr.Markdown("# AI Summarizer Chatbot")
    gr.Markdown("Enter a URL to extract text, images, and summaries. Optionally, add 'caption: [your prompt]' to customize image captions.")

    chatbot_interface = gr.Chatbot(label="Chat", height=500, bubble_full_width=False)
    msg = gr.Textbox(placeholder="Enter a URL or message...", show_label=False)
    submit = gr.Button("Send")

    msg.submit(chatbot, inputs=[msg, chatbot_interface], outputs=[chatbot_interface, msg])
    submit.click(chatbot, inputs=[msg, chatbot_interface], outputs=[chatbot_interface, msg])

# Launch
if __name__ == "__main__":
    iface.launch()

C:\Users\91936\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\gradio\components\chatbot.py:285: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7888

To create a public link, set `share=True` in `launch()`.
